In [1]:
!nvidia-smi

Sun Apr  2 09:09:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# transformer_crf

In [2]:
# install requirements: adaseq
!pip install transformers seqeval modelscope
!pip install adaseq --ignore-requires-python --no-deps

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 103.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 100.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 KB 34.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.2/190.2 KB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━

In [17]:
path = '/content/data/'

In [18]:
# process test file to conll format
with open(path + 'final_test.txt', 'r', encoding='utf8') as fin, \
     open(path + 'test.conll', 'w', encoding='utf8') as fout:
    for line in fin:
        guid, text = line.strip().split('\u0001')
        for token in text:
            print(token, 'O', sep=' ', file=fout)
        print('', file=fout)    

In [19]:
# prepare training configuration
from modelscope.utils.config import Config

config = Config.from_string("""
experiment:
  exp_dir: experiments/
  exp_name: transformer_crf
  seed: 42

task: named-entity-recognition

dataset:
  data_file:
    train: /content/data/train.conll
    valid: /content/data/dev.conll
    test: /content/data/test.conll
  data_type: conll

preprocessor:
  type: sequence-labeling-preprocessor
  max_length: 100

data_collator: SequenceLabelingDataCollatorWithPadding

model:
  type: sequence-labeling-model
  embedder:
    model_name_or_path: damo/nlp_raner_named-entity-recognition_chinese-base-generic
  dropout: 0.1
  use_crf: true

train:
  max_epochs: 20
  dataloader:
    batch_size_per_gpu: 32
  optimizer:
    type: AdamW
    lr: 5.0e-5
    param_groups:
      - regex: crf
        lr: 5.0e-1
  lr_scheduler:
    type: StepLR
    step_size: 4 
    gamma: 0.8
  hooks:
    - type: TensorboardHook

evaluation:
  dataloader:
    batch_size_per_gpu: 128
  metrics:
    - type: ner-metric
    - type: ner-dumper
      model_type: sequence_labeling
      dump_format: conll
""", file_format='.yaml')

In [ ]:
# initialize a trainer
import os
from adaseq.commands.train import build_trainer_from_partial_objects

work_dir = 'experiments/transformer_crf'
os.makedirs(work_dir, exist_ok=True)

trainer = build_trainer_from_partial_objects(
    config,
    work_dir=work_dir,
    seed=42,
    device='cuda:0'
)

# do training
trainer.train()

# do testing
trainer.test()

2023-04-02 09:56:56,820 - INFO - adaseq.data.dataset_manager - Will use a custom loading script: /usr/local/lib/python3.9/dist-packages/adaseq/data/dataset_builders/named_entity_recognition_dataset_builder.py
2023-04-02 09:56:56,844 - WARNING - datasets.builder - Using custom data configuration default-443a17dbaf35f518


Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset named_entity_recognition_dataset_builder downloaded and prepared to /root/.cache/huggingface/datasets/named_entity_recognition_dataset_builder/default-443a17dbaf35f518/0.0.0/db737b9bb893f20fb03d04403a30bf7c033256c212b7e9f0ebc6e9c958535c51. Subsequent calls will reuse this data.


...

2023-04-02 10:15:46,299 - INFO - modelscope - epoch(eval) [14][16]	memory: 4773, evaluation/precision: 0.8550, evaluation/recall: 0.8416, evaluation/f1: 0.8483, evaluation/accuracy: 0.8789
2023-04-02 10:16:00,755 - INFO - modelscope - epoch [15][50/277]	lr: 2.560e-05, eta: 0:06:04, iter_time: 0.289, data_load_time: 0.049, memory: 4773, loss: 0.2138
2023-04-02 10:16:15,914 - INFO - modelscope - epoch [15][100/277]	lr: 2.560e-05, eta: 0:05:54, iter_time: 0.303, data_load_time: 0.057, memory: 4773, loss: 0.1952
2023-04-02 10:16:28,475 - INFO - modelscope - epoch [15][150/277]	lr: 2.560e-05, eta: 0:05:43, iter_time: 0.251, data_load_time: 0.043, memory: 4773, loss: 0.1991
2023-04-02 10:16:41,132 - INFO - modelscope - epoch [15][200/277]	lr: 2.560e-05, eta: 0:05:33, iter_time: 0.253, data_load_time: 0.043, memory: 4773, loss: 0.2311
2023-04-02 10:16:53,014 - INFO - modelscope - epoch [15][250/277]	lr: 2.560e-05, eta: 0:05:21, iter_time: 0.238, data_load_time: 0.034, memory: 4773, loss: 0.26

Total test samples: 100%|██████████| 1970/1970 [00:06<00:00, 316.66it/s]


2023-04-02 10:17:06,890 - INFO - modelscope - epoch(eval) [15][16]	memory: 4773, evaluation/precision: 0.8519, evaluation/recall: 0.8415, evaluation/f1: 0.8467, evaluation/accuracy: 0.8783
2023-04-02 10:17:19,921 - INFO - modelscope - epoch [16][50/277]	lr: 2.560e-05, eta: 0:05:02, iter_time: 0.260, data_load_time: 0.040, memory: 4773, loss: 0.1658
2023-04-02 10:17:32,968 - INFO - modelscope - epoch [16][100/277]	lr: 2.560e-05, eta: 0:04:52, iter_time: 0.261, data_load_time: 0.049, memory: 4773, loss: 0.1824
2023-04-02 10:17:45,101 - INFO - modelscope - epoch [16][150/277]	lr: 2.560e-05, eta: 0:04:40, iter_time: 0.243, data_load_time: 0.038, memory: 4773, loss: 0.1994
2023-04-02 10:17:57,047 - INFO - modelscope - epoch [16][200/277]	lr: 2.560e-05, eta: 0:04:29, iter_time: 0.239, data_load_time: 0.035, memory: 4773, loss: 0.2426
2023-04-02 10:18:09,092 - INFO - modelscope - epoch [16][250/277]	lr: 2.560e-05, eta: 0:04:18, iter_time: 0.241, data_load_time: 0.040, memory: 4773, loss: 0.26

Total test samples: 100%|██████████| 1970/1970 [00:06<00:00, 318.99it/s]


2023-04-02 10:18:23,133 - INFO - modelscope - epoch(eval) [16][16]	memory: 4773, evaluation/precision: 0.8515, evaluation/recall: 0.8283, evaluation/f1: 0.8397, evaluation/accuracy: 0.8708
2023-04-02 10:18:35,466 - INFO - modelscope - epoch [17][50/277]	lr: 2.048e-05, eta: 0:03:59, iter_time: 0.246, data_load_time: 0.039, memory: 4773, loss: 0.1732
2023-04-02 10:18:47,716 - INFO - modelscope - epoch [17][100/277]	lr: 2.048e-05, eta: 0:03:48, iter_time: 0.245, data_load_time: 0.040, memory: 4773, loss: 0.1206
2023-04-02 10:18:59,742 - INFO - modelscope - epoch [17][150/277]	lr: 2.048e-05, eta: 0:03:37, iter_time: 0.240, data_load_time: 0.035, memory: 4773, loss: 0.1521


In [ ]:
# process pred file to conll format
import os

exp_dir = '/content/experiments/transformer_crf'

with open(f'{exp_dir}/pred.txt', 'r', encoding='utf8') as fin, \
     open('/content/outputs/baseline4.pred.txt', 'w', encoding='utf8') as fout:
    guid = 1
    tokens = []
    labels = []
    for line in fin:
        if line == '' or line == '\n':
            if tokens:
                print(guid, ''.join(tokens), ' '.join(labels), sep='\u0001', file=fout)
                guid += 1
                tokens = []
                labels = []
        else:
            splits = line.split('\t')
            tokens.append(splits[0])
            labels.append(splits[-1].rstrip())
    if tokens:
        print(guid, ''.join(tokens), ' '.join(labels), sep='\u0001', file=fout)

# transformer_crf_with_prompts

In [ ]:
# install requirements: adaseq
!pip install spacy
!pip install transformers seqeval modelscope
!pip install adaseq --ignore-requires-python --no-deps

In [ ]:
# prepare gazetteer

# data loading functions
def load_conll_file(filename):
    with open(filename, 'r', encoding='utf8') as f:
        tokens = []
        labels = []
        for line in f:
            if line == '' or line == '\n':
                if tokens:
                    yield {
                        'tokens': tokens,
                        'labels': labels
                    }
                    tokens = []
                    labels = []
            else:
                splits = line.rsplit(' ', 1)
                tokens.append(splits[0])
                labels.append(splits[-1].rstrip())
        if tokens:
            yield {
                'tokens': tokens,
                'labels': labels
            }
            
def get_entities(tokens, labels):
    entities = []
    cur_entity = {}
    for token, label in zip(tokens, labels):
        if label[0] in 'OBS' and cur_entity:
            entities.append(cur_entity)
            cur_entity = {}
        if label[0] in 'BS':
            cur_entity = {
                'text': token,
                'type': label[2:]
            }
        elif label[0] in 'IE':
            cur_entity['text'] += token
    if cur_entity:
        entities.append(cur_entity)
    return entities


# load datasets and count entities
train_dataset = list(load_conll_file('data/train.conll'))
dev_dataset = list(load_conll_file('data/dev.conll'))

from collections import Counter
entity_type_distribution = {}

for example in train_dataset + dev_dataset:
    entities = get_entities(example['tokens'], example['labels'])
    for e in entities:
        if '0' in e['text']: # filter out digits
            continue
        if len(e['text']) <= 2: 
            continue
        if e['text'] not in entity_type_distribution:
            entity_type_distribution[e['text']] = Counter()
        if e['type'] in ['prov', 'city', 'district', 'town', 'community', 'devzone', 'village_group', 'road', 'poi']: # keep only informing types
            entity_type_distribution[e['text']].update([e['type']])
        
# for each text in the gazetteer, only keep its most probable type
gazetteer = {}
threshold = 1
for text in entity_type_distribution:
    total_freq = sum(entity_type_distribution[text].values())
    if total_freq >= threshold: # only keep entities whose occurrence >= threshold
        tp = entity_type_distribution[text].most_common(1)[0][0]
        gazetteer[text] = tp
        
print('Number of gazetteer entities:', len(gazetteer))
# print(gazetteer)

# define rule-based predictor
from spacy.lang.zh import Chinese

class RuleBasedTagger:
    def __init__(self, gazetteer):
        nlp = Chinese()
        ruler = nlp.add_pipe('entity_ruler')
        patterns = [{'label': label, 'pattern': text} for text, label in gazetteer.items()]
        with nlp.select_pipes(enable='tagger'):
            ruler.add_patterns(patterns)
        self.nlp = nlp
        
    def predict(self, sentence):
        doc = self.nlp(sentence)
        return [{
                    'text': span.text,
                    'label': span.label_,
                    'start': span.start,
                    'end': span.end
                } for span in doc.ents]
    

tagger = RuleBasedTagger(gazetteer)
# print(tagger.predict('温州市宋埠公园'))

In [ ]:
# apply gazetteer to train/dev/test data as prompt

eng2chn = {
    'prov': '省',
    'city': '市',
    'district': '区县',
    'town': '镇街道',
    'community': '社区村',
    'devzone': '开发区',
    'village_group': '村',
    'road': '路',
    'poi': '兴趣点'
}

with open('data/train.with_prompt.conll', 'w', encoding='utf8') as fout:
    for data in train_dataset:
        for token, label in zip(data['tokens'], data['labels']):
            print(token, label, sep=' ', file=fout)
        # add prompt
        tagged_entities = tagger.predict(''.join(data['tokens']))
        if len(tagged_entities) > 0:
            print('<EOS>', 'X', sep=' ', file=fout)
            prompt = ''
            for e in tagged_entities:
                prompt += e['text'] + '是' + eng2chn[e['label']] + '，'
            for token in prompt:
                print(token, 'X', sep=' ', file=fout)    
        print('', file=fout)
        
with open('data/dev.with_prompt.conll', 'w', encoding='utf8') as fout:
    for data in dev_dataset:
        for token, label in zip(data['tokens'], data['labels']):
            print(token, label, sep=' ', file=fout)
        # add prompt
        tagged_entities = tagger.predict(''.join(data['tokens']))
        if len(tagged_entities) > 0:
            print('<EOS>', 'X', sep=' ', file=fout)
            prompt = ''
            for e in tagged_entities:
                prompt += e['text'] + '是' + eng2chn[e['label']] + '，'
            for token in prompt:
                print(token, 'X', sep=' ', file=fout)    
        print('', file=fout)

with open('data/final_test.txt', 'r', encoding='utf8') as fin, \
     open('data/test.with_prompt.conll', 'w', encoding='utf8') as fout:
    for line in fin:
        guid, text = line.strip().split('\u0001')
        for token in text:
            print(token, 'O', sep=' ', file=fout)
        # add prompt
        tagged_entities = tagger.predict(text)
        if len(tagged_entities) > 0:
            print('<EOS>', 'X', sep=' ', file=fout)
            prompt = ''
            for e in tagged_entities:
                prompt += e['text'] + '是' + eng2chn[e['label']] + '，'
            for token in prompt:
                print(token, 'X', sep=' ', file=fout)    
        print('', file=fout)    

In [ ]:
# prepare training configuration
from modelscope.utils.config import Config

config = Config.from_string("""
experiment:
  exp_dir: experiments/
  exp_name: transformer_crf_with_prompt
  seed: 42

task: named-entity-recognition

dataset:
  data_file:
    train: /mnt/workspace/tianchi/data/train.with_prompt.conll
    valid: /mnt/workspace/tianchi/data/dev.with_prompt.conll
    test: /mnt/workspace/tianchi/data/test.with_prompt.conll
  data_type: conll

preprocessor:
  type: sequence-labeling-preprocessor
  max_length: 150

data_collator: SequenceLabelingDataCollatorWithPadding

model:
  type: sequence-labeling-model
  embedder:
    model_name_or_path: damo/nlp_raner_named-entity-recognition_chinese-base-news
  dropout: 0.1
  use_crf: true

train:
  max_epochs: 20
  dataloader:
    batch_size_per_gpu: 16
  optimizer:
    type: AdamW
    lr: 5.0e-5
    param_groups:
      - regex: crf
        lr: 5.0e-1
  lr_scheduler:
    type: StepLR
    step_size: 2 
    gamma: 0.8
  hooks:
    - type: TensorboardHook

evaluation:
  dataloader:
    batch_size_per_gpu: 128
  metrics:
    - type: ner-metric
    - type: ner-dumper
      model_type: sequence_labeling
      dump_format: conll
""", file_format='.yaml')

In [ ]:
# initialize a trainer
import os
from adaseq.commands.train import build_trainer_from_partial_objects

work_dir = 'experiments/transformer_crf_with_prompt'
os.makedirs(work_dir, exist_ok=True)

trainer = build_trainer_from_partial_objects(
    config,
    work_dir=work_dir,
    seed=42,
    device='cuda:0'
)

# do training
trainer.train()

# do testing
trainer.test()

In [ ]:
# process test file to conll format
import os

# find prediction file
exp_dir = 'experiments/transformer_crf_with_prompt'

with open(f'{exp_dir}/pred.txt', 'r', encoding='utf8') as fin, \
     open('outputs/baseline5.pred.txt', 'w', encoding='utf8') as fout:
    guid = 1
    tokens = []
    labels = []
    is_orig = True
    for line in fin:
        if line == '' or line == '\n':
            if tokens:
                print(guid, ''.join(tokens), ' '.join(labels), sep='\u0001', file=fout)
                guid += 1
                tokens = []
                labels = []
                is_orig = True
        else:
            splits = line.split('\t')
            if splits[0] == '<EOS>':
                is_orig = False
            if is_orig:
                tokens.append(splits[0])
                labels.append(splits[-1].rstrip())
    if tokens:
        print(guid, ''.join(tokens), ' '.join(labels), sep='\u0001', file=fout)